In [1]:

import os
import pickle
import pandas as pd
import sys
import sys
from memory_profiler import profile
import pandas as pd
import numpy as np
sys.path.append('/home/ak/Documents/Research/PaperCode/MultiKernelLearning')
sys.path.append('/home/ak/Documents/Research/PaperCode/singlekernelclf/')
import jsonpickle
from collections import defaultdict
from sklearn.svm import SVC
from collections import OrderedDict
import os
from sklearn.metrics import accuracy_score
import torch


from fileutils import new_feature_utils as nfu
from fileutils.new_feature_utils import CreateMarketFeatures
import multiprocessing
import mkl_data_processing as mkldp
from matplotlib.ticker import ScalarFormatter,AutoMinorLocator
import matplotlib as mpl
import matplotlib.pyplot as plt
styleFiles = os.listdir(mpl.get_configdir())
styleFileIdx = 0

plt.style.use(os.path.join(mpl.get_configdir(), styleFiles[styleFileIdx]))

In [2]:
def open_pickle_filepath(pickle_file):
    pickle_to_file = pickle.load(open(pickle_file, "rb"), encoding='latin1')

    return pickle_to_file

In [3]:
files_location = '/media/ak/DataOnly/ExperimentCommonLocs/MKLOOSPredictions/SimpleMKL/'
symbols = sorted(os.listdir(files_location))
#processed results path
processed_location = '/media/ak/DataOnly/ExperimentCommonLocs/MKLOOSProcessed'

In [147]:
symbolIDX =  6#pick a symbol
label_idx = 2 #pick a label
label_string= 'Label_idx:_'+str(label_idx)
print('you are doing symbol', symbols[symbolIDX], 'and label', label_string)
# now we take out the files for the symbol 
symbol_path = os.path.join(files_location,str(sorted(symbols)[symbolIDX]))
symbolFiles = os.listdir(os.path.join(files_location,str(sorted(symbols)[symbolIDX])))
# now isolate all the label+ symbol combos
symbolLabelFiles = [f for f in symbolFiles if str(label_string) in f]
path_to_store = os.path.join(processed_location, symbols[symbolIDX])
number_of_files = len(symbolLabelFiles) 

# this is the number of files that correspond to the label/symbol combo

print('total files for the symbol:', len(symbolFiles))
print('and for this label')
print(number_of_files)

you are doing symbol BLT.L and label Label_idx:_2
total files for the symbol: 3304
and for this label
416


In [124]:
path_to_store

'/media/ak/DataOnly/ExperimentCommonLocs/MKLOOSProcessed/ITV.L'

In [125]:
dates_chunks_keys = dict()
chunks_list =list()
keys_list = list()

for file_to_load_idx in range(0,number_of_files):
    # go through all of the files that are a combination of symbols and labels
    file_to_load_path = os.path.join(os.path.join(files_location, str(symbols[symbolIDX])), symbolLabelFiles[file_to_load_idx])
    if os.path.getsize(file_to_load_path)> 0:
        # extract the key - these are the symbol feature path keys in the  main code
        file_keys = list(open_pickle_filepath(file_to_load_path).keys()) # keys
        keys_list.append(file_keys[0])
        dates_chunks_keys[file_to_load_idx] = file_keys
        chunks_list.append(open_pickle_filepath(file_to_load_path)[file_keys[0]])
    else:
        continue

        

In [126]:
dates_df = pd.DataFrame((np.unique(list(dates_chunks_keys.values()))))
list_of_dfs_across_days = list()
for chunk_list_item in range(0,len(chunks_list)-1):
    chunk_list_item_dict = chunks_list[chunk_list_item]
    chunk_list_item_keys = list(chunk_list_item_dict.keys())
    list_of_dicts=list()
    for key in chunk_list_item_keys:
        list_of_dicts.append(pd.DataFrame(chunk_list_item_dict[key]))
    list_of_dfs_across_days.append(pd.concat(list_of_dicts,axis=1).median(axis=1))


In [127]:
df =pd.concat(list_of_dfs_across_days, axis =1)
df.median(axis =1)

Hamming Loss    0.12
accuracy        0.88
f1- macro       0.47
f1- micro       0.88
f1- weighted    0.83
precision       0.78
recall          0.88
dtype: float64

In [128]:
#df.to_csv('file_name.csv
location_for_df = os.path.join(path_to_store, symbols[symbolIDX] +"_Label_"+str(label_idx)+"_all_chunks.csv")
df.to_csv(location_for_df, index= True)
location_for_dates_df = os.path.join(path_to_store, symbols[symbolIDX] +"_Label_"+str(label_idx)+"_dates.csv")
dates_df.to_csv(location_for_dates_df, index = True)

In [129]:
df_test = pd.read_csv(location_for_df, index_col=0)
df_test.median(axis=1)

Hamming Loss    0.12
accuracy        0.88
f1- macro       0.47
f1- micro       0.88
f1- weighted    0.83
precision       0.78
recall          0.88
dtype: float64

# to do:
1. write a loop to go through all the labels for each symbol and store them in a seperate file
2. concentrate only on the symbols we know we have labels
3. pull the file from Single Kernel and compare them.
4. re-do the labels and the plots for the symbols that you are picking and describe them

In [133]:
row_labels = ['accuracy','precision','recall','f1- weighted','f1- micro','f1- macro',
              'Hamming Loss']

In [134]:

# Path
def create_path(parent_dir, symbol):
    path = os.path.join(parent_dir, symbol)
    if os.path.isdir(path)==False:
        # Create the directory
    
        os.mkdir(path)
        print("Directory '%s' created" %path)
    else:
        print("Directory Exist")
        pass

In [132]:
symbols_to_load = os.listdir(processed_location)

In [135]:
list_files = list()
string = str('_Label_1')
for idx in range(0,40):
    files =os.listdir(os.path.join(processed_location, symbols_to_load[idx]))
    list_files.append([f for f in files if string in f])

        

In [136]:
list_files

[[],
 [],
 [],
 ['BLT.L_Label_1_dates.csv', 'BLT.L_Label_1_all_chunks.csv'],
 ['KGF.L_Label_1_dates.csv', 'KGF.L_Label_1_all_chunks.csv'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['RDSa.L_Label_1_dates.csv', 'RDSa.L_Label_1_all_chunks.csv'],
 [],
 [],
 [],
 [],
 [],
 ['ITV.L_Label_1_dates.csv', 'ITV.L_Label_1_all_chunks.csv'],
 [],
 [],
 ['APF.L_Label_1_dates.csv', 'APF.L_Label_1_all_chunks.csv'],
 [],
 ['REL.L_Label_1_all_chunks.csv', 'REL.L_Label_1_dates.csv'],
 ['AZN.L_Label_1_dates.csv', 'AZN.L_Label_1_all_chunks.csv'],
 ['CCL.L_Label_1_all_chunks.csv', 'CCL.L_Label_1_dates.csv'],
 [],
 [],
 ['LLOY.L_Label_1_all_chunks.csv', 'LLOY.L_Label_1_dates.csv'],
 [],
 [],
 ['CPG.L_Label_1_dates.csv', 'CPG.L_Label_1_all_chunks.csv'],
 ['BARC.L_Label_1_all_chunks.csv', 'BARC.L_Label_1_dates.csv'],
 ['AV.L_Label_1_all_chunks.csv', 'AV.L_Label_1_dates.csv'],
 [],
 [],
 [],
 [],
 []]

In [137]:
label_1_files = ['BARC.L_Label_1_all_chunks.csv',
    'APF.L_Label_1_all_chunks.csv', 
                 'ITV.L_Label_1_all_chunks.csv',
                 'CCL.L_Label_1_all_chunks.csv',
                 'AV.L_Label_1_all_chunks.csv',
                'BLT.L_Label_1_all_chunks.csv',
                'AZN.L_Label_1_all_chunks.csv',
                'KGF.L_Label_1_all_chunks.csv',
                'CPG.L_Label_1_all_chunks.csv',
                'RDSa.L_Label_1_all_chunks.csv',
                'REL.L_Label_1_all_chunks.csv',
                'LLOY.L_Label_1_all_chunks.csv']

label_5_files = ['PSON.L_Label_5_all_chunks.csv', 
                 'PRU.L_Label_5_all_chunks.csv',
                 'AV.L_Label_5_all_chunks.csv', 
                 'APF.L_Label_5_all_chunks.csv', 
                 'MKS.L_Label_5_all_chunks.csv']

label_4_files =  ['NG.L_Label_4_all_chunks.csv',
  'BLT.L_Label_4_all_chunks.csv',
  'KGF.L_Label_4_all_chunks.csv', 
  'PRU.L_Label_4_all_chunks.csv',
  'RTO.L_Label_4_all_chunks.csv',
 'RR.L_Label_4_all_chunks.csv',
'SHP.L_Label_4_all_chunks.csv',
 'RSA.L_Label_4_all_chunks.csv',
'CEY.L_Label_4_all_chunks.csv',
'RB.L_Label_4_all_chunks.csv',
'ITV.L_Label_4_all_chunks.csv',
'PSON.L_Label_4_all_chunks.csv',
  'SDR.L_Label_4_all_chunks.csv',
  'APF.L_Label_4_all_chunks.csv',
'REL.L_Label_4_all_chunks.csv', 
'AZN.L_Label_4_all_chunks.csv',
'CCL.L_Label_4_all_chunks.csv',
 'SGE.L_Label_4_all_chunks.csv', 
 'RDSb.L_Label_4_all_chunks.csv', 
 'CPG.L_Label_4_all_chunks.csv',
'AV.L_Label_4_all_chunks.csv',
'MAB.L_Label_4_all_chunks.csv',
  'LAND.L_Label_4_all_chunks.csv', 
'LGEN.L_Label_4_all_chunks.csv']

label_2_files = ['BLT.L_Label_2_all_chunks.csv', 
                 'KGF.L_Label_2_all_chunks.csv',
                'CEY.L_Label_2_all_chunks.csv',
                 'CPG.L_Label_2_all_chunks.csv',
                 'BARC.L_Label_2_all_chunks.csv',
                 'ITV.L_Label_2_all_chunks.csv',
                 'APF.L_Label_2_all_chunks.csv' ,
                'LAND.L_Label_2_all_chunks.csv',
                'LGEN.L_Label_2_all_chunks.csv',
                 'MAB.L_Label_2_all_chunks.csv'
                ]


label_3_files =['LGEN.L_Label_3_all_chunks.csv',
                'APF.L_Label_3_all_chunks.csv',
                'AV.L_Label_3_all_chunks.csv',
                'MAB.L_Label_3_all_chunks.csv',
                'LAND.L_Label_3_all_chunks.csv',
               'LLOY.L_Label_3_all_chunks.csv',
               'RBS.L_Label_3_all_chunks.csv']

label_6_files = ['RSA.L_Label_6_all_chunks.csv',
                'WPP.L_Label_6_all_chunks.csv',
                'MKS.L_Label_6_all_chunks.csv',
                'RR.L_Label_6_all_chunks.csv',
                'LAND.L_Label_6_all_chunks.csv',
                'CEY.L_Label_6_all_chunks.csv',
                 'CCL.L_Label_6_all_chunks.csv',
                 'KGF.L_Label_6_all_chunks.csv'
                ]


In [138]:
# more efficient way to do this
label_2_dfs = list()
label_2_dicts = dict()

for idx in range(0, len(label_2_files)):
    symbol = label_2_files[idx].split("_")[0]
    
    file_to_load = \
    pd.read_csv(os.path.join(processed_location, symbol, label_2_files[idx])).median(axis=1)
    label_2_dfs.append(file_to_load)
    file_to_load.index = row_labels
    label_2_dicts[symbol] = file_to_load
    
# more efficient way to do this
label_6_dfs = list()
label_6_dicts = dict()

for idx in range(0, len(label_6_files)):
    symbol = label_6_files[idx].split("_")[0]
    
    file_to_load = \
    pd.read_csv(os.path.join(processed_location, symbol, label_6_files[idx])).median(axis=1)
    label_6_dfs.append(file_to_load)
    file_to_load.index = row_labels
    label_6_dicts[symbol] = file_to_load
    
    
# more efficient way to do this
label_5_dfs = list()
label_5_dicts = dict()

for idx in range(0, len(label_5_files)):
    symbol = label_5_files[idx].split("_")[0]
    
    file_to_load = \
    pd.read_csv(os.path.join(processed_location, symbol, label_5_files[idx])).median(axis=1)
    label_5_dfs.append(file_to_load)
    file_to_load.index = row_labels
    label_5_dicts[symbol] = file_to_load

In [139]:
# more efficient way to do this
label_4_dfs = list()
label_4_dicts = dict()

for idx in range(0, len(label_4_files)):
    symbol = label_4_files[idx].split("_")[0]
    
    file_to_load = \
    pd.read_csv(os.path.join(processed_location, symbol, label_4_files[idx])).median(axis=1)
    label_4_dfs.append(file_to_load)
    file_to_load.index = row_labels
    label_4_dicts[symbol] = file_to_load

In [140]:
pd.concat(label_6_dicts, axis =1).median(axis=1)

accuracy        0.540
precision       0.460
recall          0.220
f1- weighted    0.460
f1- micro       0.320
f1- macro       0.285
Hamming Loss    0.460
dtype: float64

In [141]:
pd.concat(label_1_dfs, axis =1).median(axis=1)

Hamming Loss    0.02
accuracy        0.98
f1- macro       0.49
f1- micro       0.98
f1- weighted    0.97
precision       0.96
recall          0.98
dtype: float64

In [142]:
pd.concat(label_6_dfs, axis =1)

,0,1,2,3,4,5,6,7
accuracy,0.54,0.57,0.53,0.53,0.55,0.51,0.55,0.54
precision,0.46,0.43,0.47,0.47,0.45,0.49,0.45,0.46
recall,0.22,0.24,0.22,0.22,0.26,0.22,0.23,0.22
f1- weighted,0.46,0.43,0.47,0.47,0.45,0.49,0.45,0.46
f1- micro,0.30,0.32,0.32,0.30,0.37,0.32,0.33,0.32
f1- macro,0.27,0.37,0.26,0.25,0.41,0.24,0.37,0.30
Hamming Loss,0.46,0.43,0.47,0.47,0.45,0.49,0.45,0.46


In [143]:
pd.concat(label_4_dfs, axis =1)

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
accuracy,0.11,0.20,0.14,0.16,0.13,0.18,0.07,0.210,0.11,0.11,...,0.27,0.09,0.25,0.20,0.16,0.23,0.21,0.03,0.24,0.1775
precision,0.89,0.80,0.86,0.84,0.87,0.82,0.93,0.790,0.89,0.89,...,0.73,0.91,0.75,0.80,0.84,0.77,0.79,0.97,0.76,0.8225
recall,0.47,0.44,0.46,0.46,0.47,0.45,0.48,0.440,0.47,0.47,...,0.42,0.48,0.43,0.44,0.46,0.44,0.44,0.49,0.43,0.4500
f1- weighted,0.89,0.80,0.86,0.84,0.87,0.82,0.93,0.790,0.89,0.89,...,0.73,0.91,0.75,0.80,0.84,0.77,0.79,0.97,0.76,0.8225
f1- micro,0.84,0.71,0.80,0.77,0.82,0.74,0.89,0.700,0.84,0.84,...,0.61,0.87,0.65,0.71,0.77,0.67,0.69,0.96,0.66,0.7475
f1- macro,0.79,0.64,0.74,0.71,0.76,0.68,0.86,0.625,0.79,0.79,...,0.53,0.83,0.57,0.64,0.71,0.60,0.62,0.94,0.58,0.6800
Hamming Loss,0.89,0.80,0.86,0.84,0.87,0.82,0.93,0.790,0.89,0.89,...,0.73,0.91,0.75,0.80,0.84,0.77,0.79,0.97,0.76,0.8225


In [91]:


label_dfs = {'1':label_1_dfs,
             '2':label_2_dfs,
             '3':label_3_dfs,
             '4':label_4_dfs,
             '5':label_5_dfs,
             '6':label_6_dfs
            }

label_dicts = {'1':label_1_dicts,
             '2':label_2_dicts,
             '3':label_3_dicts,
             '4':label_4_dicts,
             '5':label_5_dicts,
             '6':label_6_dicts
            }

label_files ={'1':label_1_files,
              '2':label_2_files,
              '3':label_3_files,
              '4':label_4_files,
              '5':label_5_files,
              '6':label_6_files
             }

labelIds = list(label_dfs.keys())

In [92]:
median_dfs_across_all_symbols = dict()
for idx in labelIds:
    median_dfs_across_all_symbols['Label_'+str(idx)] = pd.DataFrame(label_dfs[str(idx)]).T.median(axis=1)

In [93]:
print(pd.DataFrame.from_dict(median_dfs_across_all_symbols).T.to_latex())

\begin{tabular}{lrrrrrrr}
\toprule
{} &  Hamming Loss &  accuracy &  f1- macro &  f1- micro &  f1- weighted &  precision &  recall \\
\midrule
Label\_1 &         0.025 &     0.975 &       0.49 &      0.975 &         0.965 &      0.955 &   0.975 \\
Label\_2 &         0.115 &     0.885 &       0.47 &      0.885 &         0.840 &      0.790 &   0.885 \\
Label\_3 &         0.620 &     0.380 &       0.26 &      0.380 &         0.310 &      0.300 &   0.380 \\
Label\_4 &         0.160 &     0.840 &       0.46 &      0.840 &         0.770 &      0.710 &   0.840 \\
Label\_5 &         0.525 &     0.475 &       0.24 &      0.475 &         0.330 &      0.290 &   0.475 \\
Label\_6 &         0.540 &     0.460 &       0.22 &      0.460 &         0.320 &      0.285 &   0.460 \\
\bottomrule
\end{tabular}



In [94]:
labelIdx = str(1)
label_Idx_files = label_files[labelIdx]
label_Idx_dfs = label_dfs[labelIdx]
label_Idx_dicts = label_dicts[labelIdx]



for idx in range(0, len(label_Idx_files)):
    symbol = label_Idx_files[idx].split("_")[0]
    
    file_to_load = \
    pd.read_csv(os.path.join(processed_location, symbol, label_Idx_files[idx])).median(axis=1)
    label_Idx_dfs.append(file_to_load)
    file_to_load.index = row_labels
    label_Idx_dicts[symbol] = file_to_load

In [77]:
label_Idx_dicts

{'BARC.L': Hamming Loss    0.01
 accuracy        0.99
 f1- macro       0.50
 f1- micro       0.99
 f1- weighted    0.98
 precision       0.98
 recall          0.99
 dtype: float64, 'APF.L': Hamming Loss    0.18
 accuracy        0.82
 f1- macro       0.45
 f1- micro       0.82
 f1- weighted    0.74
 precision       0.67
 recall          0.82
 dtype: float64, 'CCL.L': Hamming Loss    0.03
 accuracy        0.97
 f1- macro       0.49
 f1- micro       0.97
 f1- weighted    0.96
 precision       0.95
 recall          0.97
 dtype: float64, 'AV.L': Hamming Loss    0.03
 accuracy        0.97
 f1- macro       0.49
 f1- micro       0.97
 f1- weighted    0.96
 precision       0.95
 recall          0.97
 dtype: float64, 'BLT.L': Hamming Loss    0.01
 accuracy        0.99
 f1- macro       0.50
 f1- micro       0.99
 f1- weighted    0.99
 precision       0.98
 recall          0.99
 dtype: float64, 'AZN.L': Hamming Loss    0.01
 accuracy        0.99
 f1- macro       0.50
 f1- micro       0.99
 f1- wei

In [66]:
labels_dicts_processed = {'One': pd.DataFrame.from_dict(label_Idx_dicts['1']),
               'Two':  pd.DataFrame.from_dict(label_Idx_dicts['2']),
               'Three':  pd.DataFrame.from_dict(label_Idx_dicts['3']),
               'Four':  pd.DataFrame.from_dict(label_Idx_dicts['4']),
               'Five':  pd.DataFrame.from_dict(label_Idx_dicts['5']),
               'Six':  pd.DataFrame.from_dict(label_Idx_dicts['6']),}

KeyError: '1'

In [69]:
symbols_for_label_one = list(labels_dicts_processed['One'].columns.values)
symbols_for_label_two = list(labels_dicts_processed['Two'].columns.values)
symbols_for_label_three = list(labels_dicts_processed['Three'].columns.values)
symbols_for_label_four = list(labels_dicts_processed['Four'].columns.values)
symbols_for_label_five = list(labels_dicts_processed['Five'].columns.values)
symbols_for_label_six = list(labels_dicts_processed['Six'].columns.values)

common_symbols = set(symbols_for_label_one)  - set(symbols_for_label_five) - \
set(symbols_for_label_four)


NameError: name 'labels_dicts_processed' is not defined

In [25]:
print(list(common_symbols))

NameError: name 'common_symbols' is not defined

In [26]:
df = pd.DataFrame()
for key in list(labels_dicts_processed.keys()):
    df[key] =labels_dicts_processed[str(key)].median(axis =1)

NameError: name 'labels_dicts_processed' is not defined

In [779]:
print(df.to_latex())

\begin{tabular}{lrrrrrr}
\toprule
{} &   One &    Two &  Three &  Four &   Five &    Six \\
\midrule
Hamming Loss &  0.02 &  0.115 &   0.62 &  0.16 &  0.525 &  0.535 \\
accuracy     &  0.98 &  0.885 &   0.38 &  0.84 &  0.475 &  0.465 \\
f1- macro    &  0.49 &  0.470 &   0.26 &  0.46 &  0.240 &  0.220 \\
f1- micro    &  0.98 &  0.885 &   0.38 &  0.84 &  0.475 &  0.465 \\
f1- weighted &  0.97 &  0.840 &   0.31 &  0.77 &  0.330 &  0.320 \\
precision    &  0.96 &  0.790 &   0.30 &  0.71 &  0.290 &  0.265 \\
recall       &  0.98 &  0.885 &   0.38 &  0.84 &  0.475 &  0.465 \\
\bottomrule
\end{tabular}

